In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
import glob
import os
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [3]:
# import our chat-bot intents file
import json
with open('data/intents.json') as json_data:
    intents = json.load(json_data)

files = glob.glob('models/*') #delete the models folder content
for f in files:
    os.remove(f)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

111 documents
31 classes ['Action movies:', 'Adventure movies:', 'Animation movies:', 'Arabic:', 'Chinese:', 'Comedy movies:', 'Documentary movies:', 'Drama movies:', 'Easter Eggs', 'English', 'Family movies:', 'Fantasy movies:', 'French:', 'Genres:', 'German:', 'Highest Grossing Movies:', 'History movies:', 'Horror movies:', 'Italian:', 'Japanese:', 'Korean:', 'Most popular movies:', 'Music movies:', 'Mystery movies:', 'Portuguese:', 'Romance movies:', 'Russian:', 'Science fiction movies:', 'Some movies in Hindi:', 'Spanish', 'Thriller movies:']
84 unique stemmed words [',', 'act', 'admir', 'adv', 'anim', 'ar', 'arab', 'as', 'best', 'cartoon', 'categ', 'childr', 'chin', 'chines', 'comedy', 'demand', 'desir', 'docu', 'dram', 'earn', 'east', 'eg', 'engl', 'famy', 'fantasy', 'favo', 'favourit', 'fi', 'fict', 'film', 'frant', 'french', 'funny', 'genr', 'germ', 'germany', 'gross', 'had', 'highest', 'hind', 'hist', 'hor', 'in', 'ind', 'is', 'it', 'ita', 'jap', 'japanes', 'known', 'kor', 'la

In [5]:
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [6]:
# reset underlying graph data
tf.compat.v1.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=420, batch_size=8, show_metric=True)
model.save('models/model.tflearn')

Training Step: 5879  | total loss: 2.03680 | time: 0.012s
| Adam | epoch: 420 | loss: 2.03680 - acc: 0.8617 -- iter: 104/111
Training Step: 5880  | total loss: 1.83664 | time: 0.013s
| Adam | epoch: 420 | loss: 1.83664 - acc: 0.8755 -- iter: 111/111
--
INFO:tensorflow:/Users/danielkrasovski/Documents/GitHub/Project_Oreo/models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "models/training_data", "wb" ) )

